In [81]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [82]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl

Merges

In [83]:
# Function to assign tiers
def categorize_city(number_of_residents):
    if number_of_residents > 1000000:
        return 'Tier 1'
    elif number_of_residents > 400000:
        return 'Tier 2'
    elif number_of_residents > 100000:
        return 'Tier 3'
    else:
        return 'Tier 4'
    
# Read Population CSV from Wikipedia

# fix errors in data instead of on bad-lines-skip
df_city = pd.read_csv('./data/population.csv', delimiter=",", usecols=range(10))

# Extract necessary data
df_city = df_city[['Name', '2022', 'Bundesland']]
df_city['2022'] = df_city['2022'].str.replace('.', '')
df_city['2022'] = pd.to_numeric(df_city['2022'], errors='coerce')

# Assign Tiers by calling the function
df_city['City Tier'] = df_city['2022'].apply(categorize_city)

total_city_count = df_city.groupby('City Tier')['2022'].count()


# Read vendor data csv
df_vendor = pd.read_csv('./data/vendors.csv', delimiter=",", low_memory=False)

# Extract relevant data
df_vendor_subset = df_vendor[['Vendor ID', 'Vendor Name Platform', 'Brand', 'City', 'Region', 'Legal Entity Name']]

df_city = df_city.rename(columns={'2022': 'Population', 'Name': 'Vendor_City'})

# Merge Vendor Data with Tiers
df_merged = pd.merge(df_city, df_vendor_subset, left_on='Vendor_City', right_on='City')
# inplace=True -> modify original df, axis=1 -> column (row -> axis=0)
# df_merged.drop('City', axis=1, inplace=True)

# Read food order csv
df_orders = pd.read_csv('./data/food_orders.csv', delimiter=",", low_memory=False)

# Extract relevant data
df_orders_subset = df_orders[['Vendor ID','Vendor Name','Order ID','Ordered At','Vendor Region','Brand','Order Source Name','Order Source Type', 'Fulfilment Type', 'Gmv', 'Rating Food', 'Rating Delivery', 'Vouchers Total Value Gross']]

# Merge Vendor Data with Tiers through Vendor Data <> Tier DF
df_orders_per_vendor = pd.merge(df_merged, df_orders_subset, on='Vendor ID')

Calculations

In [84]:
# City Metrics

# No. of orders per city
city_order_counts = (df_orders_per_vendor.groupby('Vendor_City')['Order ID'].count()).astype(float)

# No. of vendors per city
vendor_counts = df_orders_per_vendor.groupby('Vendor_City')['Vendor ID'].nunique()

# Population per City
population_per_city = df_city.set_index('Vendor_City')['Population']

# Avg. Order Volume per Resident per City
city_avg_order_volume_per_resident = (city_order_counts / population_per_city).dropna()

# Avg. Food Rating per City
avg_food_rating_per_city = df_orders_per_vendor.groupby('Vendor_City')['Rating Food'].mean()

# Avg. Delivery Rating per City
avg_delivery_rating_per_city = df_orders_per_vendor.groupby('Vendor_City')['Rating Delivery'].mean()


# Generate CSV

df_city_metrics = pd.DataFrame({
    'Total No. of cities': total_city_count,
    'Total No. of orders': city_order_counts,
    'Total No. of vendors': vendor_counts,
    'Population': population_per_city,
    'Avg. Order Volume per Resident per City': city_avg_order_volume_per_resident,
    'Avg. Food Rating': avg_food_rating_per_city,
    'Avg. Delivery Rating': avg_delivery_rating_per_city
})

# Merging df city and metrics df to include the City Tier
df_city_metrics.reset_index(inplace=True)
df_city_metrics.rename(columns={'index': 'City'}, inplace=True)
df_city_metrics = df_city_metrics.merge(df_city[['Vendor_City', 'City Tier']], left_on='City', right_on='Vendor_City', how='left')
df_city_metrics.drop('Vendor_City', axis=1, inplace=True)
print(df_city_metrics.head())

df_city_metrics.to_csv('city_metrics.csv', index=False)

     City  Total No. of cities  Total No. of orders  Total No. of vendors  \
0  Aachen                  NaN                  NaN                   NaN   
1   Aalen                  NaN                  NaN                   NaN   
2  Achern                  NaN                  NaN                   NaN   
3   Achim                  NaN                  NaN                   NaN   
4   Ahaus                  NaN                  NaN                   NaN   

   Population  Avg. Order Volume per Resident per City  Avg. Food Rating  \
0    252136.0                                      NaN               NaN   
1     68816.0                                      NaN               NaN   
2     26471.0                                      NaN               NaN   
3     32752.0                                      NaN               NaN   
4     40245.0                                      NaN               NaN   

   Avg. Delivery Rating City Tier  
0                   NaN    Tier 3  
1       

In [85]:
# Vendor Metrics

# No. of Orders per Vendor
total_orders_per_vendor = df_orders_per_vendor.groupby('Vendor ID')['Order ID'].count().sort_values(ascending=False)

# AOV per vendor
avg_order_value_per_vendor = df_orders_per_vendor.groupby('Vendor ID')['Gmv'].mean()

# Avg. food rating per vendor
avg_food_rating_per_vendor = df_orders_per_vendor.groupby('Vendor ID')['Rating Food'].mean()

# Avg. delivery rating per vendor
avg_delivery_rating_per_vendor = df_orders_per_vendor.groupby('Vendor ID')['Rating Delivery'].mean()

# Avg. daily orders per vendor
df_orders_per_vendor['Ordered At'] = pd.to_datetime(df_orders_per_vendor['Ordered At'], utc=True).dt.date
unique_operating_days_per_vendor = df_orders_per_vendor.groupby('Vendor ID')['Ordered At'].nunique()

vendor_first_order_date = pd.to_datetime(df_orders_per_vendor.groupby('Vendor ID')['Ordered At'].min())
vendor_last_order_date = pd.to_datetime(df_orders_per_vendor.groupby('Vendor ID')['Ordered At'].max())
avg_daily_orders_per_vendor = total_orders_per_vendor / unique_operating_days_per_vendor


df_vendor_metrics = pd.DataFrame({
    'Total No. of Orders': total_orders_per_vendor,
    'AOV': avg_order_value_per_vendor,
    'Avg. Food Rating': avg_food_rating_per_vendor,
    'Avg. Delivery Rating': avg_delivery_rating_per_vendor,
    'Days operating': unique_operating_days_per_vendor,
    'Avg. Daily orders': avg_daily_orders_per_vendor,
    'First order date': vendor_first_order_date,
    'Last order date': vendor_last_order_date
})

df_vendor_metrics.reset_index(inplace=True)
# Generate CSV

df_vendor_metrics.to_csv('vendor_metrics.csv', index=False)



In [86]:
# Tier metrics

# Total No. of orders per Tier
tier_order_counts = df_orders_per_vendor.groupby('City Tier')['Order ID'].count().astype(float)

# AOV per Tier
aov_per_tier = df_orders_per_vendor.groupby('City Tier')['Gmv'].mean()

# Total GMV per Tier
total_gmv_per_tier = df_orders_per_vendor.groupby('City Tier')['Gmv'].sum()

# Total Population per Tier

# list of unique cities where orders have been placed -> filter df_city for these values
unique_cities_with_orders = df_orders_per_vendor['Vendor_City'].unique()
df_city_with_orders = df_city[df_city['Vendor_City'].isin(unique_cities_with_orders)]
total_population_served_per_tier = df_city_with_orders.groupby('City Tier')['Population'].sum().astype(float)

# Avg. Food Rating per Tier
avg_food_rating_per_tier = df_orders_per_vendor.groupby('City Tier')['Rating Food'].mean()

# Avg. Delivery Rating per Tier
avg_delivery_rating_per_tier = df_orders_per_vendor.groupby('City Tier')['Rating Delivery'].mean()

# No. of different fulfilment types per Tier
# total_no_own_delivery = df_orders_per_vendor.groupby('Fulfilment Type')['Order ID'].count()

# No. of unique Vendors per Tier
count_vendors_per_tier = df_orders_per_vendor.groupby('City Tier')['Vendor ID'].nunique()

# No. of served cities
served_cities = df_orders_per_vendor.groupby('City Tier')['Vendor_City'].nunique()

# Avg. Daily Orders per Vendor per Tier

vendor_tiers = df_orders_per_vendor[['Vendor ID', 'City Tier']].drop_duplicates()

df_avg_daily_orders = pd.merge(avg_daily_orders_per_vendor.reset_index().rename(columns={0: 'Avg. Daily Orders'}), vendor_tiers, on='Vendor ID')

avg_daily_orders_per_vendor_per_tier = df_avg_daily_orders.groupby('City Tier')['Avg. Daily Orders'].mean()

# Avg. Daily Order Volume per Resident Per Tier (Count of orders/Population in active cities)
tier_avg_order_volume_per_resident = tier_order_counts / total_population_served_per_tier

# Avg. Daily Order Volume per Resident (Daily Order counts per Tier / Population in active cities)
avg_daily_order_volume_per_resident_per_tier = (avg_daily_orders_per_vendor_per_tier / total_population_served_per_tier)


# Combine all metrics into a single DataFrame
df_tier_metrics = pd.DataFrame({
    'Total No. of orders': tier_order_counts,
    'AOV': aov_per_tier,
    'Total GMV': total_gmv_per_tier,
    'Total Population Served': total_population_served_per_tier,
    'Avg. Order Volume per Resident': tier_avg_order_volume_per_resident,
    'Avg. Daily Order Volume per Resident': avg_daily_order_volume_per_resident_per_tier,
    # 'No. of different fulfilment types': total_no_own_delivery,
    'No. of unique Vendors': count_vendors_per_tier,
    'Avg. Daily Orders per Vendor': avg_daily_orders_per_vendor_per_tier,
    'Avg. food rating': avg_food_rating_per_tier,
    'Avg. delivery rating': avg_delivery_rating_per_tier
})

In [87]:
# Overall metrics

# Total No. of Orders
total_orders = df_orders_per_vendor['Order ID'].count()

# Avg. No. of Orders
avg_number_of_orders_per_vendor = df_orders_per_vendor.groupby('Vendor ID')['Order ID'].count().mean()

# No. of unique Vendors
count_vendors = df_orders_per_vendor['Vendor ID'].nunique()

# Total GMV
total_gmv = total_gmv_per_tier

df_metrics_general = pd.Series({
    'Total No. of Orders': total_orders,
    'Avg. No. of Orders per Vendor': avg_number_of_orders_per_vendor,
    'No. of unique Vendors': count_vendors,
})

# Reset the index
df_tier_metrics.reset_index(inplace=True)

# Export to CSV

df_metrics_general.to_csv('general_metrics.csv', index=False)
df_tier_metrics.to_csv('tier_metrics_30.csv', index=False)

In [88]:
# Avg. Daily Orders per Vendor over time

# df_orders_timestamp = df_orders_per_vendor[['Vendor ID', 'Ordered At', 'City Tier']]

# df_orders_timestamp['Date'] = pd.to_datetime(df_orders_timestamp['Ordered At'], utc=True).dt.date

# # Group, then count no. of orders for each group
# daily_orders = df_orders_timestamp.groupby(['Date', 'Vendor ID', 'City Tier']).size()
# daily_orders = daily_orders.reset_index(name='Orders')

# avg_daily_orders = daily_orders.groupby(['Date', 'City Tier'])['Orders'].mean()

# # Reset the index of avg_daily_orders to turn it back into a DataFrame
# avg_daily_orders = avg_daily_orders.reset_index(name='Avg. Orders')

# avg_daily_orders.to_csv('daily_aov_over_time_per_vendor.csv', index=False)


In [98]:
# Extract weeks
df_orders_per_vendor['Ordered At'] = pd.to_datetime(df_orders_per_vendor['Ordered At'])
df_orders_per_vendor['Week'] = df_orders_per_vendor['Ordered At'].dt.to_period('W')

# Group by Week, Tiers -> count unique vendors
weekly_orders = df_orders_per_vendor.groupby(['Week', 'City Tier'])['Vendor ID'].nunique().reset_index(name='Vendor Count')

# Total no. of vendors for each week
weekly_totals = weekly_orders.groupby('Week')['Vendor Count'].sum()

# Proportion of vendors per tier per week
weekly_orders['Vendor Proportion'] = weekly_orders.apply(lambda row: row['Vendor Count'] / weekly_totals[row['Week']], axis=1)

# weekly_orders.to_csv('weekly_orders.csv', index=False)



df_orders_per_vendor['']


Week
2023-05-22/2023-05-28     50
2023-05-29/2023-06-04     48
2023-06-05/2023-06-11     45
2023-06-12/2023-06-18     43
2023-06-19/2023-06-25     49
2023-06-26/2023-07-02     52
2023-07-03/2023-07-09     49
2023-07-10/2023-07-16     49
2023-07-17/2023-07-23     50
2023-07-24/2023-07-30     52
2023-07-31/2023-08-06     54
2023-08-07/2023-08-13     64
2023-08-14/2023-08-20     65
2023-08-21/2023-08-27     72
2023-08-28/2023-09-03     70
2023-09-04/2023-09-10     75
2023-09-11/2023-09-17     79
2023-09-18/2023-09-24     76
2023-09-25/2023-10-01     76
2023-10-02/2023-10-08     76
2023-10-09/2023-10-15    133
2023-10-16/2023-10-22    134
2023-10-23/2023-10-29    130
2023-10-30/2023-11-05    123
2023-11-06/2023-11-12    121
2023-11-13/2023-11-19    125
2023-11-20/2023-11-26    119
2023-11-27/2023-12-03    115
2023-12-04/2023-12-10    114
2023-12-11/2023-12-17    115
2023-12-18/2023-12-24    120
2023-12-25/2023-12-31    117
2024-01-01/2024-01-07    117
2024-01-08/2024-01-14    118
2024-01-1